## Preprocess

In [1]:
import pandas as pd

In [2]:
# Загрузка исходного XLSX файла
input_file = '/home/jupyter/datasphere/project/v2_ragas_npa_dataset_firstPart.xlsx'  # Укажите путь к вашему файлу
df = pd.read_excel(input_file)

In [3]:
df['question']

0      Каков объем экспорта услуг категории "Поездки"...
1      Каковы были основные мероприятия, реализуемые ...
2      Каков размер финансирования из федерального бю...
3      Какой комплексный центр социального обслуживан...
4      Какое значение имеет экологическое образование...
                             ...                        
495                Что входит в расходы госорганов Югры?
496    Как изменился объем переработки газа в 2009 году?
497    Что дает программа "Развитие материально-техни...
498            Какую долю занимают средства Нефтеюганск?
499    Субвенции ХМАО-Югры: как распределяются и что ...
Name: question, Length: 500, dtype: object

In [4]:
pattern = r"^\['(.*)'\]$"

df['contexts'] = df['contexts'].str.replace(pattern, r'\1', regex=True)

In [5]:
df['contexts'].head()

0    Увеличение объема экспорта услуг категории "По...
1    и | |развития | | |Ханты-Мансийского |исполнит...
2    не относящиеся к районам Крайнего Севера и при...
3    |2.2. |Центр социальной |240 к/мест |7580,0 |2...
4    (полного)общего образования и начального профе...
Name: contexts, dtype: object

In [6]:
answer = 'Оценка релевантности: набор фрагментов документов релевантен, так как в них содержатся ключевые слова и семантические значения, связанные с вопросом пользователя о реформах, связанных с ветеранами труда.\n\n**Ключевые слова:** ветераны труда, ветераны Великой Отечественной войны, ветераны боевых действий, труженики тыла, меры социальной поддержки, присвоение звания «ветеран труда», льготы, субсидии, законы «О ветеранах», «О мерах социальной поддержки отдельных категорий граждан».\n\nВ представленных фрагментах документов описывается законодательство и меры поддержки ветеранов труда и других категорий граждан, а также изменения и дополнения к этим законам. Это указывает на наличие информации о возможных реформах в этой области. Однако конкретные сведения о введении новых реформ или изменениях в законодательстве требуют более детального анализа и выделения конкретных цитат из фрагментов.'

## IMPORTS

In [7]:
%pip install ragas

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [8]:
from ragas.metrics import faithfulness, answer_relevancy, context_recall, context_precision
from ragas import evaluate 

In [9]:
from datasets import Dataset
from ragas.metrics import faithfulness, answer_correctness

In [10]:
from ragas.llms import LangchainLLMWrapper

In [11]:
%pip install yandexcloud

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [12]:
from langchain_core.language_models import BaseLanguageModel
from langchain_core.embeddings import Embeddings
from langchain_community.chat_models import ChatYandexGPT
from langchain_community.llms import YandexGPT

In [32]:
llm = YandexGPT(iam_token='', folder_id='b1g5pkij4170jm1a1un6', temperature=0)

In [14]:
%pip install -U langchain_community tiktoken langchain-openai langchainhub langchain langgraph langchain_huggingface
%pip install -U pymilvus sentence-transformers fsspec s3fs yandexcloud transformers pip install langchain-experimental
%pip install --upgrade huggingface-hub
%pip install -qU  langchain_milvus
%pip install pydantic==2.6.0
%pip install ragas langchain_openai python-dotenv

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement install (from versions: none)

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip
ERROR: No matching distribution found for install
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.26.4 which is incomp

## RAGAS

In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_milvus import Milvus
from langchain_huggingface import HuggingFaceEmbeddings
from transformers import AutoModel

model = AutoModel.from_pretrained("deepvk/USER-bge-m3", trust_remote_code=True) 

model_name = "deepvk/USER-bge-m3"
model_kwargs = {'device': 'cpu'}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
)


USER_AGENT environment variable not set, consider setting it to identify your requests.
/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2024-09-13 08:36:38.835207: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-13 08:36:38.888678: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-13 08:36:39.909475: W tensorflow/compiler/tf2tensorrt/util

In [16]:
langchain_llm = LangchainLLMWrapper(langchain_llm=llm)

In [17]:
import re
import string

# questions for retrieval
questions = {
    "question": "Чем обновлять школы округа?"
}

dataset_text = ''.join([char.lower() if not char.isdigit() and char is not None else char for char in questions["question"]])
dataset_text = re.sub('  ', ' ', dataset_text) # remove useless space
dataset_text = re.sub(r'[\x00-\x1F\x7F-\x9F]+', '', dataset_text)

# questions["question"] = dataset_text
dataset_text

'чем обновлять школы округа?'

In [18]:
# Вопрос от вашей RAG-системы
rag_question = dataset_text

# Удаление лишних пробелов
df['question'] = df['question'].str.strip()
rag_question = rag_question.strip()

# Поиск вопроса в базе данных
matched_row = df[df['question'] == questions["question"]]

# Проверяем, найден ли вопрос
if not matched_row.empty:
    # Извлекаем правильный ответ и контекст
    ground_truth = matched_row['ground_truth'].values[0]
    contexts = matched_row['contexts'].values[0]
    # Ответ от вашей RAG-системы (замените на реальный ответ)
    rag_answer = "Муниципалитету следует обратиться к законодательству и выяснить, может ли он взять на себя функции регионального оператора в случае его бездействия. Также можно обратиться в соответствующие органы власти для получения рекомендаций и поддержки в данной ситуации."

    data_samples = {
        "question": [
            dataset_text[:100]
        ],
        "answer": [
            rag_answer[:100]
        ],
        "contexts": [
            [contexts[:100]]
        ],
        "ground_truth": [
            ground_truth[:100]
        ],
    }

    # print(rag_question)
    # print(contexts)
    # print(ground_truth)
    # print(rag_answer)
    
    # Создаем Dataset для оценки
    dataset = Dataset.from_dict(data_samples)

else:
    print("Вопрос не найден в базе данных.")
    
# dataset = Dataset.from_pandas(df.iloc[:10])
# dataset

In [19]:
dataset.to_pandas().head()

,question,answer,contexts,ground_truth
0,чем обновлять школы округа?,Муниципалитету следует обратиться к законодате...,[|3. |Обновление |Департамент |получение всеми...,Школы округа следует обновлять следующим образ...


In [20]:
import pandas as pd

# Путь к файлу Excel
input_file = '/home/jupyter/datasphere/s3/hack-object-storage/database/file_for_merge.xlsx'

# Чтение файла Excel и создание DataFrame
df_2 = pd.read_excel(input_file)

# Вывод DataFrame
print(df_2)

                                            contexts  ...                                             answer
0  по оснащению образовательного процесса, реализ...  ...  Учебно-наглядные пособия и оборудованные кабин...
1  для перевозки обучающихся 0,000 0,000 0,000 3 ...  ...  Учебно-наглядные пособия и оборудованные кабин...

[2 rows x 3 columns]


In [21]:
# Преобразование Dataset в DataFrame
dataset_df = dataset.to_pandas()

# Объединение df_2 и dataset по колонке 'question'
merged_df = pd.merge(dataset_df, df_2[['question', 'answer', 'contexts']], on='question', how='left', suffixes=('', '_new'))

# Замена значений колонок answer и contexts
merged_df['answer'] = merged_df['answer_new']
merged_df['contexts'] = merged_df['contexts_new']

# Удаление временных колонок
merged_df = merged_df.drop(columns=['answer_new', 'contexts_new'])

In [25]:
# Проверка типа данных в колонке 'contexts'
types = merged_df['contexts'].apply(type).unique()
print(types)

[<class 'str'>]


In [26]:
merged_df['contexts'] = merged_df['contexts'].apply(lambda x: x.split(',') if isinstance(x, str) else [])

In [28]:
merged_df

,question,answer,contexts,ground_truth
0,чем обновлять школы округа?,Учебно-наглядные пособия и оборудованные кабин...,"[по оснащению образовательного процесса, реал...",Школы округа следует обновлять следующим образ...
1,чем обновлять школы округа?,Учебно-наглядные пособия и оборудованные кабин...,"[для перевозки обучающихся 0, 000 0, 000 0, 00...",Школы округа следует обновлять следующим образ...


In [29]:
# Преобразование DataFrame в Dataset
updated_dataset = Dataset.from_pandas(merged_df)

# Вывод информации о Dataset для проверки
print(updated_dataset)

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truth', '__index_level_0__'],
    num_rows: 2
})


In [35]:
from ragas.metrics import (answer_relevancy,faithfulness,context_recall,context_precision)
from ragas.metrics.critique import harmfulness
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
import ragas

evaluation_chat_model = LangchainLLMWrapper(llm)
evaluation_embeddings = LangchainEmbeddingsWrapper(embeddings)

def evaluate_ragas_dataset(ragas_dataset):
    run_config = ragas.RunConfig(timeout=1000, max_retries=1, max_wait=10, max_workers = 1, log_tenacity = True)
    result = evaluate(
        updated_dataset,  # Run without the last row
        metrics=[context_precision,faithfulness,answer_relevancy,context_recall],
        llm=llm,
        embeddings=embeddings,
        raise_exceptions=True,
        run_config=run_config,
    )
    return result.to_pandas()


result = evaluate_ragas_dataset(updated_dataset)
result

Evaluating: 100%|██████████| 8/8 [02:57<00:00, 22.19s/it]
Task exception was never retrieved
future: <Task finished name='Task-9' coro=<as_completed.<locals>.sema_coro() done, defined at /home/jupyter/.local/lib/python3.10/site-packages/ragas/executor.py:32> exception=AssertionError('LLM is not set')>
Traceback (most recent call last):
  File "/usr/lib/python3.10/asyncio/tasks.py", line 232, in __step
    result = coro.send(None)
  File "/home/jupyter/.local/lib/python3.10/site-packages/ragas/executor.py", line 34, in sema_coro
    return await coro
  File "/home/jupyter/.local/lib/python3.10/site-packages/ragas/executor.py", line 60, in wrapped_callable_async
    raise e
  File "/home/jupyter/.local/lib/python3.10/site-packages/ragas/executor.py", line 54, in wrapped_callable_async
    result = await callable(*args, **kwargs)
  File "/home/jupyter/.local/lib/python3.10/site-packages/ragas/metrics/base.py", line 246, in single_turn_ascore
    raise e
  File "/home/jupyter/.local/lib/py

,question,contexts,answer,ground_truth,context_precision,faithfulness,answer_relevancy,context_recall
0,чем обновлять школы округа?,"[по оснащению образовательного процесса, реал...",Учебно-наглядные пособия и оборудованные кабин...,Школы округа следует обновлять следующим образ...,0.00000,0.0,0.0,0.5
1,чем обновлять школы округа?,"[для перевозки обучающихся 0, 000 0, 000 0, 00...",Учебно-наглядные пособия и оборудованные кабин...,Школы округа следует обновлять следующим образ...,0.22619,0.0,0.0,0.5


In [ ]:
%pip freeze > requirements.txt